In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import seaborn as sns 
import numpy as np 
from tqdm import tqdm
import random
import datetime as dt

In [ ]:
PATH = '/Users/krc/Desktop/project/system_quality_project/data/'

tre = pd.read_csv(PATH + 'train_err.csv')
tse = pd.read_csv(PATH + 'test_err.csv')

# tre.time = pd.to_datetime(tre.time.astype('str'))
# tre.time 
# tse.head()

In [ ]:
def make_date(x):
    x = str(x)
    year = int(x[0:4])
    month = int(x[4:6])
    day = int(x[6:8])
    hour = int(x[8:10])
    minute = int(x[10:12])
    sec = int(x[12:])
    return dt.datetime(year,month,day,hour,minute,sec)
    
# tse['datetime'] = tse.time.apply(make_date)
# tse

In [ ]:
tre.time = pd.to_datetime(tre.time)

In [ ]:
# tse.drop(columns='time').to_csv(PATH + 'test_err.csv',index= False)

In [ ]:
tp = pd.read_csv(PATH + 'train_problem_data.csv')
tp.time = tp.time.apply(make_date)
p_user = tp.user_id.unique()
# p_user, set(p_user)
np_user = list(set(tre.user_id.unique()) -set(p_user))

In [ ]:
tq = pd.read_csv(PATH + 'train_quality_data.csv')

tqq = tq[['time','user_id']].drop_duplicates().reset_index(drop = True)
tqq.time = tqq.time.apply(make_date)
# tqq 시간과 user_id 만 쓰네 ;

/var/folders/t3/j8459bg97zb7x34h0c_jgnlr0000gn/T/ipykernel_2248/3691190635.py:1: DtypeWarning: Columns (10,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  tq = pd.read_csv(PATH + 'train_quality_data.csv')


In [ ]:
tqq

,time,user_id
0,2020-11-29 09:00:00,10000
1,2020-11-30 21:00:00,10000
2,2020-11-04 11:00:00,10002
3,2020-11-06 01:00:00,10002
4,2020-11-11 01:00:00,10002
...,...,...
68947,2020-11-30 20:30:00,24993
68948,2020-11-28 20:20:00,24995
68949,2020-11-29 00:20:00,24995
68950,2020-11-08 23:00:00,24997


In [ ]:
# tpd=tp.user_id.value_counts().to_frame().reset_index()
# tpd.columns = ['user_id','count_e'] # columns 명을 이렇게도 바꿀수있구만
# tpd
# 컬러명 가져오기?

def check_unique(col,df1,df2):
    def change_len(x):
        if len(x) == 10:
            return x[:5]
        else:
            return x
    print('about',col)
    if col == 'fwver':
        train_c = set(df1[col].apply(change_len).unique())
        test_c = set(df2[col].apply(change_len).unique())
        print("train only in %s" %col,train_c -test_c)
        print("test only in %s"%col,test_c - train_c)
         # 차집합 개념이구나
        total = (train_c | test_c)
        
    else:
        train_c = set(df1[col].unique())
        test_c = set(df2[col].unique())
        print("train only in %s" %col,train_c -test_c)
        print("test only in %s"%col,test_c - train_c)
         # 차집합 개념이구나
        total = (train_c | test_c)
        # 합집합
        
model_total=check_unique('model_nm',tre,tse)
errtype_total=check_unique('errtype',tre,tse)
fwver_total = check_unique('fwver',tre,tse)

about model_nm
train only in model_nm set()
test only in model_nm set()
about errtype
train only in errtype set()
test only in errtype set()
about fwver
train only in fwver set()
test only in fwver {'10.22'}


In [ ]:
%%time
def change_len(x):
        if len(x) == 10:
            return x[:5]
        else:
            return x
set(tre['fwver'].apply(change_len).unique())

CPU times: user 1.89 s, sys: 138 ms, total: 2.02 s
Wall time: 2.02 s


{'03.11',
 '04.16',
 '04.22',
 '04.33',
 '04.73',
 '04.82',
 '05.15',
 '05.66',
 '10',
 '8.5.3'}

In [ ]:
%%time
set(tre['fwver'])

CPU times: user 619 ms, sys: 1.77 ms, total: 620 ms
Wall time: 620 ms


{'03.11.1141',
 '03.11.1149',
 '03.11.1167',
 '04.16.2641',
 '04.16.3345',
 '04.16.3439',
 '04.16.3553',
 '04.16.3569',
 '04.16.3571',
 '04.22.1442',
 '04.22.1656',
 '04.22.1666',
 '04.22.1684',
 '04.22.1750',
 '04.22.1778',
 '04.33.1095',
 '04.33.1125',
 '04.33.1149',
 '04.33.1171',
 '04.33.1185',
 '04.33.1261',
 '04.73.2237',
 '04.73.2571',
 '04.82.1684',
 '04.82.1730',
 '04.82.1778',
 '05.15.2090',
 '05.15.2092',
 '05.15.2114',
 '05.15.2120',
 '05.15.2122',
 '05.15.2138',
 '05.15.3104',
 '05.66.3237',
 '05.66.3571',
 '10',
 '8.5.3'}

In [ ]:
pd.options.display.max_columns = 50
pd.options.display.max_rows = 200

In [ ]:
t1 = tre.loc[tre.errtype==1]
        
t11 = t1.user_id.value_counts().describe().to_frame().T.rename(index ={'user_id':1})
pnumber = len(t1.loc[t1.user_id.isin(p_user)].user_id.unique())
npnumber = len(t1.loc[t1.user_id.isin(np_user)].user_id.unique())

t11['p_counts'] = pnumber
print(round((pnumber/1)*100,2))
t11

24200.0


,count,mean,std,min,25%,50%,75%,max,p_counts
1,950.0,22.645263,18.925383,1.0,2.0,30.0,30.0,171.0,242


In [ ]:
len(tre.loc[(tre.errtype==1) & (tre.loc[tre.errtype==1].user_id.isin(p_user))])/1#/1

4014.0

In [ ]:
def get_distribution_errtype():
    total = []
    for num in range(1,43):
        if num == 29:
            continue
        t1 = tre.loc[tre.errtype==num]
        
        t11 = t1.user_id.value_counts().describe().to_frame().T.rename(index ={'user_id':num})
        pnumber = len(t1.loc[t1.user_id.isin(p_user)].user_id.unique())
        npnumber = len(t1.loc[t1.user_id.isin(np_user)].user_id.unique())
        
        t11['p_counts'] = pnumber
        t11['npnumber'] = npnumber
        # if pnumber+npnumber == 0:
            # t11['p_ratio'] =  round((pnumber/1)*100,2)
        # else:
        t11['p_ratio'] = round((pnumber/(pnumber+npnumber))*100,2)
        total.append(t11)
    return pd.concat(total).reset_index().rename(columns={'index':'errtype'})

errtype_distribution = get_distribution_errtype()

errtype_distribution

,errtype,count,mean,std,min,25%,50%,75%,max,p_counts,npnumber,p_ratio
0,1,950.0,22.645263,18.925383,1.0,2.0,30.0,30.0,171.0,242,708,25.47
1,2,390.0,66.920513,352.584466,1.0,2.0,2.0,6.0,4056.0,168,222,43.08
2,3,1679.0,18.163788,208.580954,1.0,1.0,2.0,4.0,7239.0,465,1214,27.70
3,4,6881.0,187.761372,2774.102374,1.0,1.0,16.0,151.0,195024.0,2298,4583,33.40
4,5,11010.0,57.246140,163.044126,1.0,3.0,8.0,35.0,4207.0,4028,6982,36.58
5,6,6467.0,7.798516,47.282503,1.0,1.0,2.0,4.0,1796.0,2662,3805,41.16
6,7,7648.0,6.795502,46.745625,1.0,1.0,2.0,4.0,2687.0,3093,4555,40.44
7,8,64.0,2.453125,1.851037,1.0,1.0,2.0,3.0,12.0,13,51,20.31
8,9,63.0,1.730159,2.444258,1.0,1.0,1.0,1.0,17.0,38,25,60.32
9,10,2360.0,29.166102,236.123915,1.0,1.0,2.0,6.0,7311.0,1134,1226,48.05


In [ ]:
tre[tre.errtype == 29]

,user_id,time,model_nm,fwver,errtype,errcode


In [ ]:
def make(num):
    err =  tre.loc[tre.errtype==num]
    users = err.user_id.unique()
    # users2 = err.user_id.value_counts().to_frame().index # 굳이 이렇게 하나 싶어서 바꿈
    # t2 = err.loc[err.user_id.isin(users)]
    err2 = err.copy()
    err2['err_'+str(num)] = err2.time.dt.hour # datatime 객체로 되있는거에서 year hour등등 만 뽑을라면 .dt.뽑으려는것
    # 으로 해야되더라고요 판다스 series 에서 시간 처리하는 메소드 였네요 근데 이건 저장했다가 불려오면그냥 str로 읽어 오는게되서
    # 다시한번 date time으로 바꿔 주는 과정이 필요한것 같아요 실험 해보야 되여
    
    p = err2.loc[err2.user_id.isin(p_user)] # 만들어놓은 problem의 유저랑
    np = err2.loc[err2.user_id.isin(np_user)] # problem 접수안한 유저로  또 구분해준다 각각 에러타입 별
    
    display(p)
    pdf = p['err_'+str(num)].value_counts().sort_index().to_frame().T.rename(index = {'err_'+str(num):\
                                                                                      'err_p'+str(num)})
    npdf = np['err_'+str(num)].value_counts().sort_index().to_frame().T.rename(index = {'err_'+str(num):\
                                                                                      'err_np'+str(num)})
    # 위에서 에러 발생 hour 을 컬럼으로 만들어줘서
    # 밑에 보이는 것처럼 transpose해서 err_type 이 어느 시간에 얼마나 발생했는지 볼수있게만들었고 
    # 그걸 또 불만접수 여부로 나누었다
    
    # f는 불만접수 여부 나눈 dataframe을 합친것이다
    f = pd.concat([pdf,npdf])
    display(f)
    f2 = f.copy()
    f = f.fillna(0)
    
    f = round(f)
    # 뭔가 소수점이 있었나본데 값이
    # 굳이 copy해서 두개를 뽑은 이유는 모름
    display(f.info())
    return f, f2
# make라는 함수를 만들어서 매개변수로 errtype 숫자들을 받아 errtype 별로 데이터 프레임 만들고
make(1)



,user_id,time,model_nm,fwver,errtype,errcode,err_1
168129,10168,2020-11-01 02:17:02,model_4,03.11.1167,1,0,2
168151,10168,2020-11-02 05:18:05,model_4,03.11.1167,1,0,5
168209,10168,2020-11-04 04:26:36,model_4,03.11.1167,1,0,4
168214,10168,2020-11-05 03:49:02,model_4,03.11.1167,1,0,3
168227,10168,2020-11-06 02:43:56,model_4,03.11.1167,1,0,2
...,...,...,...,...,...,...,...
15365110,24990,2020-11-28 19:56:22,model_4,03.11.1167,1,0,19
15365117,24990,2020-11-29 03:02:20,model_4,03.11.1167,1,0,3
15365120,24990,2020-11-29 18:19:52,model_4,03.11.1167,1,0,18
15365139,24990,2020-11-30 07:10:49,model_4,03.11.1167,1,0,7


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
err_p1,13,8,903,908,879,542,18,40,79,35,45,27,24,32,28,43,48,43,49,63,51,67,43,26
err_np1,78,40,3971,3874,4168,2058,123,146,154,151,157,165,170,173,243,245,237,215,261,259,199,156,150,106


<class 'pandas.core.frame.DataFrame'>
Index: 2 entries, err_p1 to err_np1
Data columns (total 24 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   0       2 non-null      int64
 1   1       2 non-null      int64
 2   2       2 non-null      int64
 3   3       2 non-null      int64
 4   4       2 non-null      int64
 5   5       2 non-null      int64
 6   6       2 non-null      int64
 7   7       2 non-null      int64
 8   8       2 non-null      int64
 9   9       2 non-null      int64
 10  10      2 non-null      int64
 11  11      2 non-null      int64
 12  12      2 non-null      int64
 13  13      2 non-null      int64
 14  14      2 non-null      int64
 15  15      2 non-null      int64
 16  16      2 non-null      int64
 17  17      2 non-null      int64
 18  18      2 non-null      int64
 19  19      2 non-null      int64
 20  20      2 non-null      int64
 21  21      2 non-null      int64
 22  22      2 non-null      int64
 23  23      2 non

None

(         0   1     2     3     4     5    6    7    8    9    10   11   12  \
 err_p1   13   8   903   908   879   542   18   40   79   35   45   27   24   
 err_np1  78  40  3971  3874  4168  2058  123  146  154  151  157  165  170   
 
           13   14   15   16   17   18   19   20   21   22   23  
 err_p1    32   28   43   48   43   49   63   51   67   43   26  
 err_np1  173  243  245  237  215  261  259  199  156  150  106  ,
          0   1     2     3     4     5    6    7    8    9    10   11   12  \
 err_p1   13   8   903   908   879   542   18   40   79   35   45   27   24   
 err_np1  78  40  3971  3874  4168  2058  123  146  154  151  157  165  170   
 
           13   14   15   16   17   18   19   20   21   22   23  
 err_p1    32   28   43   48   43   49   63   51   67   43   26  
 err_np1  173  243  245  237  215  261  259  199  156  150  106  )

In [ ]:
t = []

for v in range(1,43): # 왜 v라고 했는지 모르겠지만 errtype의 숫자들을 하나씩 뽑는 for 문
    if v == 29:
        pass
    else:
        
        df ,_ = make(v)
        t.append(df)
        


,user_id,time,model_nm,fwver,errtype,errcode,err_1
168129,10168,2020-11-01 02:17:02,model_4,03.11.1167,1,0,2
168151,10168,2020-11-02 05:18:05,model_4,03.11.1167,1,0,5
168209,10168,2020-11-04 04:26:36,model_4,03.11.1167,1,0,4
168214,10168,2020-11-05 03:49:02,model_4,03.11.1167,1,0,3
168227,10168,2020-11-06 02:43:56,model_4,03.11.1167,1,0,2
...,...,...,...,...,...,...,...
15365110,24990,2020-11-28 19:56:22,model_4,03.11.1167,1,0,19
15365117,24990,2020-11-29 03:02:20,model_4,03.11.1167,1,0,3
15365120,24990,2020-11-29 18:19:52,model_4,03.11.1167,1,0,18
15365139,24990,2020-11-30 07:10:49,model_4,03.11.1167,1,0,7


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
err_p1,13,8,903,908,879,542,18,40,79,35,45,27,24,32,28,43,48,43,49,63,51,67,43,26
err_np1,78,40,3971,3874,4168,2058,123,146,154,151,157,165,170,173,243,245,237,215,261,259,199,156,150,106


<class 'pandas.core.frame.DataFrame'>
Index: 2 entries, err_p1 to err_np1
Data columns (total 24 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   0       2 non-null      int64
 1   1       2 non-null      int64
 2   2       2 non-null      int64
 3   3       2 non-null      int64
 4   4       2 non-null      int64
 5   5       2 non-null      int64
 6   6       2 non-null      int64
 7   7       2 non-null      int64
 8   8       2 non-null      int64
 9   9       2 non-null      int64
 10  10      2 non-null      int64
 11  11      2 non-null      int64
 12  12      2 non-null      int64
 13  13      2 non-null      int64
 14  14      2 non-null      int64
 15  15      2 non-null      int64
 16  16      2 non-null      int64
 17  17      2 non-null      int64
 18  18      2 non-null      int64
 19  19      2 non-null      int64
 20  20      2 non-null      int64
 21  21      2 non-null      int64
 22  22      2 non-null      int64
 23  23      2 non

None

,user_id,time,model_nm,fwver,errtype,errcode,err_2
137239,10134,2020-11-23 19:59:30,model_3,05.15.2138,2,1,19
137240,10134,2020-11-23 19:59:38,model_3,05.15.2138,2,0,19
281971,10306,2020-11-21 01:27:29,model_0,04.22.1750,2,1,1
281972,10306,2020-11-21 01:27:30,model_0,04.22.1750,2,0,1
281973,10306,2020-11-21 01:27:57,model_0,04.22.1750,2,1,1
...,...,...,...,...,...,...,...
15078804,24779,2020-11-02 19:55:11,model_2,04.33.1185,2,0,19
15324580,24951,2020-11-26 22:51:25,model_0,04.22.1778,2,1,22
15324581,24951,2020-11-26 22:51:26,model_0,04.22.1778,2,0,22
15340821,24966,2020-11-27 19:59:25,model_1,04.16.3553,2,1,19


...

None

,user_id,time,model_nm,fwver,errtype,errcode,err_42
931,10001,2020-11-21 02:24:11,model_2,04.33.1261,42,3,2
3955,10004,2020-11-27 03:14:27,model_0,04.22.1778,42,3,3
3997,10004,2020-11-29 03:22:42,model_0,04.22.1778,42,3,3
4658,10005,2020-11-27 04:48:41,model_0,04.22.1778,42,3,4
4702,10005,2020-11-29 03:52:17,model_0,04.22.1778,42,3,3
...,...,...,...,...,...,...,...
15367004,24997,2020-11-21 03:58:00,model_0,04.22.1778,42,3,3
15367056,24997,2020-11-23 03:13:36,model_0,04.22.1778,42,3,3
15367129,24997,2020-11-25 03:01:11,model_0,04.22.1778,42,3,3
15367187,24997,2020-11-27 03:12:28,model_0,04.22.1778,42,3,3


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
err_p42,6,3,3009,3230,3444,1748,4,7,21,20,22,16,20,15,17,18,26,34,43,42,45,43,27,26
err_np42,11,7,6016,6223,6667,3109,6,11,12,20,16,19,14,11,12,20,20,29,34,43,40,30,22,19


<class 'pandas.core.frame.DataFrame'>
Index: 2 entries, err_p42 to err_np42
Data columns (total 24 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   0       2 non-null      int64
 1   1       2 non-null      int64
 2   2       2 non-null      int64
 3   3       2 non-null      int64
 4   4       2 non-null      int64
 5   5       2 non-null      int64
 6   6       2 non-null      int64
 7   7       2 non-null      int64
 8   8       2 non-null      int64
 9   9       2 non-null      int64
 10  10      2 non-null      int64
 11  11      2 non-null      int64
 12  12      2 non-null      int64
 13  13      2 non-null      int64
 14  14      2 non-null      int64
 15  15      2 non-null      int64
 16  16      2 non-null      int64
 17  17      2 non-null      int64
 18  18      2 non-null      int64
 19  19      2 non-null      int64
 20  20      2 non-null      int64
 21  21      2 non-null      int64
 22  22      2 non-null      int64
 23  23      2 n

None

In [ ]:
tt = pd.concat(t).fillna(0)

tv = tt.T.corr()

tv1 = tv.loc[tv.index.str.contains('np')].iloc[:,::2]# iloc 로 컬럼들 problem 에대한 컬럼만 뽑는것
# print(tt.shape)
# [첫번째 인덱스: 마지막인덱스 :step] 으로 보면 된다 
d = []
for v in range(41):
    if v == 28:
        d.append(0)
        vv = tv1.iloc[v,v] # 하드 코드 느낌 하나라도 바뀌면 아주 아무것도 못써먹어
                            # 29번쨰 를 0부터 시작하니  28로 써 채워준것이다
    else:
        vv = tv1.iloc[v,v]
    d.append(vv)
display(d)
# 각각 불만 접수한사람과 안한사람 의 에러타입별 상관관계를 구한것이다
# 위 에 v== 28을 한이유가 밑에 꺼 돌아가게 하려고 만든것이다.
codf = pd.DataFrame(d).reset_index().rename(columns={'index':'errtype',0:'n_np_cor_hour_counts'})
codf['errtype']=codf['errtype']+1
codf
# 음이것으로 상관계수 식을 모르니 24시간을 나눠서 그걸로 상관계수를 구한게 어떻게 관계성이 높은지를 알수있게 되는지 
# 이해를 몬하것노 안되는거같에 근데 되겠지 
# 띠용 값이 다름 어떻게 값이 다를수가있지 뭐지 ?

[0.9962318654094494,
 0.3965072990350326,
 0.30877958922149706,
 0.7689093392844164,
 0.9718122825183706,
 -0.2351894551256699,
 0.4410490394571588,
 -0.012185275909275876,
 0.5668927848036025,
 0.4362701696875558,
 0.9986299135995451,
 0.9984941587998754,
 0.8300146419927472,
 -0.4811403172512371,
 0.9593349731626296,
 0.9812569589103155,
 0.9715329917736556,
 0.8700232456069996,
 0.5195870148719355,
 0.8970742799716042,
 0.5058393875420183,
 0.8083220422827995,
 0.9563215411007204,
 0.7105704135342897,
 0.5059645986395894,
 0.9970996166198509,
 0.4573749932967047,
 0.4683280112495824,
 0,
 0.7497884386042063,
 0.9716590919096628,
 0.9160931108589663,
 0.9992740153075972,
 0.9429166981001016,
 0.8346598596393029,
 0.9966073138350372,
 0.9964984408301591,
 0.9492348167332397,
 0.8888908413077122,
 0.9653425714140392,
 0.9512698394555559,
 0.9994818217551728]

,errtype,n_np_cor_hour_counts
0,1,0.996232
1,2,0.396507
2,3,0.308780
3,4,0.768909
4,5,0.971812
5,6,-0.235189
6,7,0.441049
7,8,-0.012185
8,9,0.566893
9,10,0.436270


In [ ]:
pd.merge(codf,errtype_distribution)

,errtype,n_np_cor_hour_counts,count,mean,std,min,25%,50%,75%,max,p_counts,npnumber,p_ratio
0,1,0.996232,950.0,22.645263,18.925383,1.0,2.0,30.0,30.0,171.0,242,708,25.47
1,2,0.396507,390.0,66.920513,352.584466,1.0,2.0,2.0,6.0,4056.0,168,222,43.08
2,3,0.308780,1679.0,18.163788,208.580954,1.0,1.0,2.0,4.0,7239.0,465,1214,27.70
3,4,0.768909,6881.0,187.761372,2774.102374,1.0,1.0,16.0,151.0,195024.0,2298,4583,33.40
4,5,0.971812,11010.0,57.246140,163.044126,1.0,3.0,8.0,35.0,4207.0,4028,6982,36.58
5,6,-0.235189,6467.0,7.798516,47.282503,1.0,1.0,2.0,4.0,1796.0,2662,3805,41.16
6,7,0.441049,7648.0,6.795502,46.745625,1.0,1.0,2.0,4.0,2687.0,3093,4555,40.44
7,8,-0.012185,64.0,2.453125,1.851037,1.0,1.0,2.0,3.0,12.0,13,51,20.31
8,9,0.566893,63.0,1.730159,2.444258,1.0,1.0,1.0,1.0,17.0,38,25,60.32
9,10,0.436270,2360.0,29.166102,236.123915,1.0,1.0,2.0,6.0,7311.0,1134,1226,48.05


In [ ]:
errtype_distribution

,errtype,count,mean,std,min,25%,50%,75%,max,p_counts,npnumber,p_ratio
0,1,950.0,22.645263,18.925383,1.0,2.0,30.0,30.0,171.0,242,708,25.47
1,2,390.0,66.920513,352.584466,1.0,2.0,2.0,6.0,4056.0,168,222,43.08
2,3,1679.0,18.163788,208.580954,1.0,1.0,2.0,4.0,7239.0,465,1214,27.70
3,4,6881.0,187.761372,2774.102374,1.0,1.0,16.0,151.0,195024.0,2298,4583,33.40
4,5,11010.0,57.246140,163.044126,1.0,3.0,8.0,35.0,4207.0,4028,6982,36.58
5,6,6467.0,7.798516,47.282503,1.0,1.0,2.0,4.0,1796.0,2662,3805,41.16
6,7,7648.0,6.795502,46.745625,1.0,1.0,2.0,4.0,2687.0,3093,4555,40.44
7,8,64.0,2.453125,1.851037,1.0,1.0,2.0,3.0,12.0,13,51,20.31
8,9,63.0,1.730159,2.444258,1.0,1.0,1.0,1.0,17.0,38,25,60.32
9,10,2360.0,29.166102,236.123915,1.0,1.0,2.0,6.0,7311.0,1134,1226,48.05
